In [ ]:
!pip install pandas
!pip install pillow
!pip install kagglehub
!pip install scikit-learn
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install timm

In [ ]:
import os
import random
from sklearn.metrics import accuracy_score
from PIL import Image
from itertools import combinations
import kagglehub
import torch
import torch.nn as nn
from torch.amp import GradScaler, autocast
import timm
from torchvision import models, transforms
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
def create_image_pairs_labels(dataset_dir, num_impostor_pairs=1000):
    # Dictionary to organize images by user and finger type
    user_finger_dict = {}

    # Step 1: Parse filenames and organize by user ID and finger type
    for filename in os.listdir(dataset_dir):
        if filename.endswith(".BMP"):
            parts = filename.split("__")  # Splitting by double underscore
            user_num = parts[0]
            finger_type = parts[1].rsplit("_", 1)[0]  # Extracting finger info without augmentation part

            # Add to dictionary
            if (user_num, finger_type) not in user_finger_dict:
                user_finger_dict[(user_num, finger_type)] = []
            user_finger_dict[(user_num, finger_type)].append(os.path.join(dataset_dir, filename))

    image_pairs = []
    labels = []

    # Step 2: Create genuine pairs (same user, same finger)
    for _, images in user_finger_dict.items():
        if len(images) > 1:
            for img1, img2 in combinations(images, 2):
                image_pairs.append((img1, img2))
                labels.append(1)  # 1 for genuine pairs

    # Step 3: Create impostor pairs (different user or different finger)
    user_finger_keys = list(user_finger_dict.keys())
    num_pairs = 0

    while num_pairs < num_impostor_pairs:
        user_finger1, user_finger2 = random.sample(user_finger_keys, 2)

        # Ensure different user or finger type
        if user_finger1[0] != user_finger2[0] or user_finger1[1] != user_finger2[1]:
            img1 = random.choice(user_finger_dict[user_finger1])
            img2 = random.choice(user_finger_dict[user_finger2])
            image_pairs.append((img1, img2))
            labels.append(-1)  # -1 for impostor pairs
            num_pairs += 1

    return image_pairs, labels

In [ ]:
# Custom Dataset for Siamese Network
class SiameseFingerprintDataset(Dataset):
    def __init__(self, image_pairs, labels, transform=None):
        self.image_pairs = image_pairs
        self.labels = labels
        self.transform = transform if transform else transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
        ])

    def __getitem__(self, index):
        # Load images from file paths
        img1_path, img2_path = self.image_pairs[index]
        img1 = Image.open(img1_path).convert("RGB")  # Ensure 3 channels
        img2 = Image.open(img2_path).convert("RGB")

        # Apply transformations
        img1 = self.transform(img1)
        img2 = self.transform(img2)

        # Get label
        label = torch.tensor(self.labels[index], dtype=torch.float32)

        return img1, img2, label

    def __len__(self):
        return len(self.image_pairs)

In [ ]:
class SiameseDeiTTiny(nn.Module):
    def __init__(self, embedding = 64):
        super(SiameseDeiTTiny, self).__init__()
        self.deit = timm.create_model('deit_tiny_patch16_224', pretrained=True)

        # Get the number of features from the previous layer
        num_features = self.deit.head.in_features

        # Remove the original classification head
        self.deit.head = None

        # Add a new fully connected layer with the desired embedding dimension
        self.deit.head = nn.Linear(num_features, embedding)

    def forward(self, img1, img2):
        embed1 = self.deit(img1)
        embed2 = self.deit(img2)
        return embed1, embed2

In [ ]:
# Evaluation Function
def calculate_far_frr(predictions, labels, threshold=0.5):
    true_positives = ((predictions >= threshold) & (labels == 1)).sum().item()
    false_positives = ((predictions >= threshold) & (labels == 0)).sum().item()
    true_negatives = ((predictions < threshold) & (labels == 0)).sum().item()
    false_negatives = ((predictions < threshold) & (labels == 1)).sum().item()

    far = false_positives / (false_positives + true_negatives) if (false_positives + true_negatives) > 0 else 0
    frr = false_negatives / (false_negatives + true_positives) if (false_negatives + true_positives) > 0 else 0

    return far, frr

In [ ]:
# Training Function
def train(model, train_loader, criterion, optimizer, device, num_epochs=10):
    model.train()

    # Creates a GradScaler once at the beginning of training.
    scaler = GradScaler()

    for epoch in tqdm(range(num_epochs)):
        total_loss = 0.0

        for img1, img2, label in tqdm(train_loader, desc=f"Epoch {epoch + 1}", leave=False):
          img1, img2, label = img1.to(device), img2.to(device), label.to(device)
          optimizer.zero_grad()
          # Runs the forward pass with autocasting.
          with autocast(device_type=device, dtype=torch.float16):
            # Forward pass
            embed1, embed2 = model(img1, img2)
            loss = criterion(embed1, embed2, label.float())

          # Backward pass and optimization
          scaler.scale(loss).backward()
          scaler.step(optimizer)
          scaler.update()

          total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

# Testing Function
def test_model(model, test_loader, device, threshold=0.5):
    model.eval()
    embeddings1 = []
    embeddings2 = []
    labels = []

    with torch.no_grad():
        for img1, img2, label in  tqdm(test_loader, leave=False):
            img1, img2, label = img1.to(device), img2.to(device), label.to(device)
            embed1, embed2 = model(img1, img2)
            embeddings1.append(embed1)
            embeddings2.append(embed2)
            labels.extend(label.cpu().numpy())

    embeddings1 = torch.cat(embeddings1)
    embeddings2 = torch.cat(embeddings2)
    labels = torch.tensor(labels)

    similarities = F.cosine_similarity(embeddings1, embeddings2)
    predictions = (similarities >= threshold).float()

    accuracy = accuracy_score(labels, predictions)
    far, frr = calculate_far_frr(predictions, labels, threshold)

    return accuracy, far, frr

In [ ]:
# Download latest version
path = kagglehub.dataset_download("ruizgara/socofing")

print("Path to dataset files:", path)

In [ ]:
path_easy = os.path.join(path, 'SOCOFing/Altered/Altered-Easy')
path_medium = os.path.join(path, 'SOCOFing/Altered/Altered-Medium')
path_hard = os.path.join(path, 'SOCOFing/Altered/Altered-Hard')

In [ ]:
# Prepare dataset
hard_image_pairs, hard_labels = create_image_pairs_labels(path_hard, num_impostor_pairs=20000)
medium_image_pairs, medium_labels = create_image_pairs_labels(path_medium, num_impostor_pairs=20000)
easy_image_pairs, easy_labels = create_image_pairs_labels(path_easy, num_impostor_pairs=20000)

In [ ]:
image_pairs = hard_image_pairs + medium_image_pairs + easy_image_pairs
labels = hard_labels + medium_labels + easy_labels

train_size = int(0.8 * len(image_pairs))
test_size = len(image_pairs) - train_size

train_image_pairs, test_image_pairs = image_pairs[:train_size], image_pairs[train_size:]
train_labels, test_labels = labels[:train_size], labels[train_size:]

In [ ]:
num_epochs = 5
learning_rate = 1e-4
batch_size = 512
threshold = 0.5

In [ ]:
train_dataset = SiameseFingerprintDataset(train_image_pairs, train_labels)
test_dataset = SiameseFingerprintDataset(test_image_pairs, test_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
len(train_dataset), len(test_dataset)

In [ ]:
 # Initialize Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseDeiTTiny().to(device)
criterion = nn.CosineEmbeddingLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train the model
print("Starting Training...")
train(model, train_loader, criterion, optimizer, device, num_epochs=num_epochs)

In [ ]:
# Test the model
print("Testing the model...")
accuracy, far, frr = test_model(model, test_loader, device, threshold=threshold)
print(f"Test Accuracy: {accuracy:.2f}")
print(f"False Acceptance Rate (FAR): {far:.2f}")
print(f"False Rejection Rate (FRR): {frr:.2f}")